In [1]:
import json 
import pandas as pd
import numpy as np
import datetime
import requests
from random import randint

In [2]:
# Opening JSON file 
f = open('neighbor-districts.json') 
  
# returns JSON object as a dictionary 
data = json.load(f) 

# list of districts - sample entry: pune_district/Q1797336
districts_from_json = []
for i in data:
    districts_from_json.append(i)
districts_from_json = np.array(districts_from_json)

# sorting districts alphabetically
districts_from_json.sort()

# list of district names - sample entry: pune_district
district_names_from_json = []

# list of district ids - sample entry: Q1797336
district_ids_from_json = []

for i in range(len(districts_from_json)):
    district_names_from_json.append(districts_from_json[i].split('/')[0])
    district_ids_from_json.append(districts_from_json[i].split('/')[1])

In [3]:
# modified list of district names - 
# sample modifications: pune_district -> pune, jyotiba_phule_nagar -> amroha, agar_malwa -> agar malwa
district_names_from_json_modified = []

for i in range(len(district_names_from_json)):
# from hshengar20, names of some districts have changed: https://hello.iitk.ac.in/cs685a/#/forum/general/64
    if district_names_from_json[i] == 'jyotiba_phule_nagar':
        district_names_from_json_modified.append('amroha')
    elif district_names_from_json[i] == 'sant_ravidas_nagar':
        district_names_from_json_modified.append('bhadohi')
    elif district_names_from_json[i] == 'bijapur_district':
        district_names_from_json_modified.append('vijayapura')
    elif district_names_from_json[i] == 'palghat':
        district_names_from_json_modified.append('palakkad')
    elif district_names_from_json[i] == 'faizabad':
        district_names_from_json_modified.append('ayodhya')
# discrepancies dicovered by me which need to be resolved manually
    elif district_names_from_json[i] == 'baleshwar':
        district_names_from_json_modified.append('balasore')
    elif district_names_from_json[i] == 'ysr':
        district_names_from_json_modified.append('y.s.r kadapa')
    elif district_names_from_json[i] == 'purbi_singhbhum':
        district_names_from_json_modified.append('east singhbhum')
    elif district_names_from_json[i] == 'pashchimi_singhbhum':
        district_names_from_json_modified.append('west singhbhum')
    elif district_names_from_json[i] == 'purba_champaran':
        district_names_from_json_modified.append('east champaran')
    elif district_names_from_json[i] == 'pashchim_champaran':
        district_names_from_json_modified.append('west champaran')
    elif district_names_from_json[i] == 'tirunelveli_kattabo':
        district_names_from_json_modified.append('tirunelveli')
    elif district_names_from_json[i] == 'the_nilgiris_district':
        district_names_from_json_modified.append('nilgiris')
    elif district_names_from_json[i] == 'tirunelveli_kattabo':
        district_names_from_json_modified.append('tirunelveli')
    elif district_names_from_json[i] == 'hugli':
        district_names_from_json_modified.append('hooghly')
    elif district_names_from_json[i] == 'sri_ganganagar':
        district_names_from_json_modified.append('ganganagar')
    elif district_names_from_json[i] == 'the_dangs':
        district_names_from_json_modified.append('dang')
    elif district_names_from_json[i] == 'lakhimpur':
        district_names_from_json_modified.append('lakhimpur kheri')
    elif district_names_from_json[i] == 'sahibzada_ajit_singh_nagar':
        district_names_from_json_modified.append('s.a.s. nagar')
    elif district_names_from_json[i] == 'sri_potti_sriramulu_nellore':
        district_names_from_json_modified.append('s.p.s. nellore')
    elif district_names_from_json[i] == 'muktsar':
        district_names_from_json_modified.append('sri muktsar sahib')
    elif district_names_from_json[i] == 'dantewada':
        district_names_from_json_modified.append('dakshin bastar dantewada')
    elif district_names_from_json[i] == 'kochbihar':
        district_names_from_json_modified.append('cooch behar')
    elif district_names_from_json[i] == 'sonapur':
        district_names_from_json_modified.append('subarnapur') 
    else:
        district_names_from_json_modified.append(district_names_from_json[i])
# removing "_district" from district names
    if len(district_names_from_json_modified[i])>9 and district_names_from_json_modified[i][-9:] == '_district':
        district_names_from_json_modified[i] = district_names_from_json_modified[i][:-9]
# replacing "_" by " " in district names
    for j in range(len(district_names_from_json_modified[i])):
        if district_names_from_json_modified[i][j] == '_':
            district_names_from_json_modified[i] = district_names_from_json_modified[i][:j] + ' ' + district_names_from_json_modified[i][j+1:] 

In [4]:
# dictionaries for mapping dates to time ids
# eg. for 2020-3-15, time_id_week is 1, time_id_month is 1, time_id_overall is 1

time_id_week = {}
time_id_month = {}
time_id_overall = {}
date = datetime.date(2020,3,15)
day = 1
while True:
    time_id_week[str(date)] = int(np.ceil(day/7))
    time_id_month[str(date)] = int(str(date).split('-')[1])-2
    time_id_overall[str(date)] = 1
    if date == datetime.date(2020,9,5):
        break
    day+=1
    date += datetime.timedelta(days=1)

date = datetime.date(2020,3,15)

In [5]:
# loading Covid Data into lists
r = requests.get('https://api.covid19india.org/v4/data-all.json')
dat = json.loads(r.text)

date = datetime.date(2020,3,15)

Date = []
State = []
District = []
Dist_state = []
numcases = []
weekid = []
monthid = []
overallid = []

while True:
    for state in dat[str(date)]:
        flag_districts=0
        for key in dat[str(date)][state]:
            if key == 'districts':
                flag_districts=1
                for district in dat[str(date)][state]['districts']:
                    flag_delta=0
                    for key_delta in dat[str(date)][state]['districts'][district]:
                        if key_delta == 'delta':
                            flag_delta=1
                            flag_confimed=0
                            for key_confirmed in dat[str(date)][state]['districts'][district]['delta']:
                                if key_confirmed == 'confirmed':
                                    flag_confirmed=1
                                    Date.append(str(date))
                                    State.append(state)
                                    District.append(district)
                                    Dist_state.append(district+':'+state)
                                    numcases.append(dat[str(date)][state]['districts'][district]['delta']['confirmed'])
                                    weekid.append(time_id_week[str(date)])
                                    monthid.append(time_id_month[str(date)])
                                    overallid.append(time_id_overall[str(date)])
        if flag_districts == 0:
            flag_delta=0
            if state== 'TT':
                continue
            for key in dat[str(date)][state]:
                if key == 'delta':
                    falg_delta=1
                    flag_confirmed = 0
                    for key_confirmed in dat[str(date)][state]['delta']:
                        if key_confirmed == 'confirmed':
                            flag_confirmed = 1
                            Date.append(str(date))
                            State.append(state)
                            District.append('All Districts')
                            numcases.append(dat[str(date)][state]['delta']['confirmed'])
                            weekid.append(time_id_week[str(date)])
                            monthid.append(time_id_month[str(date)])
                            overallid.append(time_id_overall[str(date)])
                            Dist_state.append('All Districts:'+state)
        
    if date == datetime.date(2020,9,5):
        break
    date += datetime.timedelta(days=1)

In [6]:
def editdist(str1, str2): 
    m = len(str1)
    n = len(str2)
    # Create a table to store results of subproblems
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)] 
  
    # Fill d[][] in bottom up manner 
    for i in range(m + 1): 
        for j in range(n + 1): 
  
            # If first string is empty, only option is to 
            # insert all characters of second string 
            if i == 0: 
                dp[i][j] = j    # Min. operations = j 
  
            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif j == 0: 
                dp[i][j] = i    # Min. operations = i 
  
            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif str1[i-1] == str2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # If last character are different, consider all 
            # possibilities and find minimum 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert 
                                   dp[i-1][j],        # Remove 
                                   dp[i-1][j-1])    # Replace 
  
    return dp[m][n] 


In [7]:
District_set = set()

for i in range(len(District)):
    District_set.add(District[i].lower() +':'+ State[i])
    
district_names_from_data_all = []
district_states_from_data_all = []
for dist in District_set:
    dis = dist.split(':')[0]
    if dis == 'all districts' or dis == 'italians' or dis == 'unknown' or dis == 'foreign evacuees' or dis == 'airport quarantine' or dis == 'railway quarantine' or dis == 'other region' or dis == 'other state' or dis == 'others' or dis =='bsf camp' :
        continue
    district_names_from_data_all.append(dis)
    district_states_from_data_all.append(dist.split(':')[1])

equivalent_district = []
equivalent_state = []
completed = np.zeros(len(district_names_from_data_all))
    
website_districts = {'District_Name': district_names_from_data_all, 'State': district_states_from_data_all, 'Completed': completed}
website_districts = pd.DataFrame(website_districts)
website_districts.sort_values("District_Name", axis = 0, ascending = True, kind='mergesort', inplace = True, na_position ='last')
website_districts = website_districts.reset_index(drop=True)


for i in range(len(district_names_from_json_modified)):
    found = 0
    for j in range(len(website_districts)):
#         if mod_sorted(district_names_from_json_modified[i])==mod_sorted(district_names_from_data_all[j]) and completed[j]==0:
        if district_names_from_json_modified[i]==website_districts['District_Name'][j] and website_districts['Completed'][j]==0:
            equivalent_district.append(website_districts['District_Name'][j])
            website_districts['Completed'][j]=1
            found = 1
            equivalent_state.append(website_districts['State'][j])
            break
    if found==0:
        equivalent_district.append('#')
        equivalent_state.append('#')

for i in range(len(district_names_from_json_modified)):
    if equivalent_district[i]=='#':
        for j in range(len(website_districts)):
            if editdist(district_names_from_json_modified[i],website_districts['District_Name'][j])<4 and website_districts['District_Name'][j][0]==district_names_from_json_modified[i][0] and website_districts['Completed'][j]==0:
                equivalent_district[i]=website_districts['District_Name'][j]
                website_districts['Completed'][j]=1
                equivalent_state[i] = website_districts['State'][j]
                break
    
json_districts = {'Names from Json': district_names_from_json, 'IDs': district_ids_from_json, 'Modified Names': district_names_from_json_modified, 'Equivalent from website': equivalent_district, 'state': equivalent_state}
json_districts = pd.DataFrame(json_districts)

# manual adjustment for districts with same name
# bilaspur
json_districts['IDs'][94], json_districts['IDs'][93] = json_districts['IDs'][93], json_districts['IDs'][94]
# balrampur
json_districts['IDs'][47], json_districts['IDs'][48] = json_districts['IDs'][48], json_districts['IDs'][47]
# hamirpur
json_districts['IDs'][230], json_districts['IDs'][231] = json_districts['IDs'][231], json_districts['IDs'][230]
# aurangabad, pratapgarh - no issues


new_districts = []
for i in range(len(website_districts)):
    if website_districts['Completed'][i]==0:
        new_districts.append(website_districts['District_Name'][i] +'/Q' + str(randint(100000, 999999)))

/home/neilrs/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/neilrs/.local/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
district_name_equivalent = {}
for i in range(len(districts_from_json)):
    if equivalent_district[i] == '#':
        district_name_equivalent[districts_from_json[i]] = equivalent_district[i]
    else:
        district_name_equivalent[districts_from_json[i]] = equivalent_district[i] + '/' + json_districts['IDs'][i]

data_modified = {}
for i in range(len(districts_from_json)):
    if district_name_equivalent[districts_from_json[i]] != '#':
        data_modified[district_name_equivalent[districts_from_json[i]]] = []
        for j in range(len(data[districts_from_json[i]])):
            if district_name_equivalent[data[districts_from_json[i]][j]]!='#':
                data_modified[district_name_equivalent[districts_from_json[i]]].append(district_name_equivalent[data[districts_from_json[i]][j]])

districts_to_complete_ids = {}
for i in range(len(equivalent_district)):
    districts_to_complete_ids[equivalent_district[i]] = equivalent_district[i] + '/' + json_districts['IDs'][i]

for i in range(len(new_districts)):
    districts_to_complete_ids[new_districts[i].split('/')[0]] = new_districts[i]

for dis in new_districts:
    data_modified[dis] = []
    
# Manual Changes to be done 

# gaurela pendra marwahi: mungeli, bilaspur, korba, koriya, anuppur | part of bilaspur: add gaurela pendra marwahi, delete anuppur, koriya
# ranipet: vellore, thiruvallur, kanchipuram, tiruvannamalai | part of kanchipuram_district
# chngalpattu: chennai, kanchipuram, tiruvannamalai, viluppuram  | part of kanchipuram_district
# Changes in kanchipuram: remove viluppuram, vellore, add ranipet, changalpattu
# hnahthial: lunglei, serchhip | part of lunglei (no change)
# khawzawl: champhai, saitual, serchhip | part of champhai
# saitual: aizawl, champhai, khawzawl, serchhip | part of champhai
# changes in champhai: remove aizwal, add saitual, khawzawl
# yanam: east godavari | Part of pondicherry (no change)
# tenkasi: tirunelvelli , virudhunagar, kollam, thiruvananthapuram, thoothukudi | part of tirunelvelli: add tenkasi, delete idukki, virudhunagar, kollam, thiruvananthapuram 
# tirupathur: krishnagiri, tiruvannamalai, vellore, chittoor | part of vellore: add ranipet, tirupathur, remove krishnagiri, kanchipuram

# mumbai: thane | combine mumbai city and mumbai suburban
# delhi: ghaziabad, gautam_buddha_nagar, gurugram, sonipat, baghpat, jhajjar, faridabad | combine 10 districts in delhi, shahdara

for key in data_modified:
    if key.split('/')[0]== 'gaurela pendra marwahi':
        data_modified[key].append(districts_to_complete_ids['mungeli'])
        data_modified[key].append('bilaspur/Q100157')
        data_modified[key].append(districts_to_complete_ids['korba'])
        data_modified[key].append(districts_to_complete_ids['koriya'])
        data_modified[key].append(districts_to_complete_ids['anuppur'])
        data_modified[districts_to_complete_ids['mungeli']].append(key)
        data_modified['bilaspur/Q100157'].append(key)
        data_modified[districts_to_complete_ids['korba']].append(key)
        data_modified[districts_to_complete_ids['koriya']].append(key)
        data_modified[districts_to_complete_ids['anuppur']].append(key)
    elif key.split('/')[0]== 'ranipet':
        data_modified[key].append(districts_to_complete_ids['vellore'])
        data_modified[key].append(districts_to_complete_ids['thiruvallur'])
        data_modified[key].append(districts_to_complete_ids['kancheepuram'])
        data_modified[key].append(districts_to_complete_ids['tiruvannamalai'])
        data_modified[districts_to_complete_ids['vellore']].append(key)
        data_modified[districts_to_complete_ids['thiruvallur']].append(key)
        data_modified[districts_to_complete_ids['kancheepuram']].append(key)
        data_modified[districts_to_complete_ids['tiruvannamalai']].append(key)
    elif key.split('/')[0]== 'tirupathur':
        data_modified[key].append(districts_to_complete_ids['krishnagiri'])
        data_modified[key].append(districts_to_complete_ids['tiruvannamalai'])
        data_modified[key].append(districts_to_complete_ids['vellore'])
        data_modified[key].append(districts_to_complete_ids['chittoor'])
        data_modified[districts_to_complete_ids['vellore']].append(key)
        data_modified[districts_to_complete_ids['krishnagiri']].append(key)
        data_modified[districts_to_complete_ids['chittoor']].append(key)
        data_modified[districts_to_complete_ids['tiruvannamalai']].append(key)
    elif key.split('/')[0]== 'chngalpattu':
        data_modified[key].append(districts_to_complete_ids['chennai'])
        data_modified[key].append(districts_to_complete_ids['kancheepuram'])
        data_modified[key].append(districts_to_complete_ids['tiruvannamalai'])
        data_modified[key].append(districts_to_complete_ids['viluppuram'])
        data_modified[districts_to_complete_ids['chennai']].append(key)
        data_modified[districts_to_complete_ids['kancheepuram']].append(key)
        data_modified[districts_to_complete_ids['tiruvannamalai']].append(key)
        data_modified[districts_to_complete_ids['viluppuram']].append(key)
    elif key.split('/')[0]== 'kancheepuram':
        for key2 in data_modified[key]:
            if key2.split('/')[0] == 'viluppuram' or key2.split('/')[0] == 'vellore':
                data_modified[key].remove(key2)
                data_modified[key2].remove(key)
    elif key.split('/')[0]== 'vellore':
        for key2 in data_modified[key]:
            if key2.split('/')[0] == 'krishnagiri':
                data_modified[key].remove(key2)
                data_modified[key2].remove(key)
    elif key.split('/')[0]== 'hnahthial':
        data_modified[key].append(districts_to_complete_ids['lunglei'])
        data_modified[key].append(districts_to_complete_ids['serchhip'])
        data_modified[districts_to_complete_ids['serchhip']].append(key)
        data_modified[districts_to_complete_ids['lunglei']].append(key)
    elif key.split('/')[0]== 'khawzawl':
        data_modified[key].append(districts_to_complete_ids['champhai'])
        data_modified[key].append(districts_to_complete_ids['saitual'])
        data_modified[key].append(districts_to_complete_ids['serchhip'])
        data_modified[districts_to_complete_ids['champhai']].append(key)
        data_modified[districts_to_complete_ids['saitual']].append(key)
        data_modified[districts_to_complete_ids['serchhip']].append(key)
    elif key.split('/')[0]== 'saitual':
        data_modified[key].append(districts_to_complete_ids['aizawl'])
        data_modified[key].append(districts_to_complete_ids['champhai'])
        data_modified[key].append(districts_to_complete_ids['khawzawl'])
        data_modified[key].append(districts_to_complete_ids['serchhip'])
        data_modified[districts_to_complete_ids['aizawl']].append(key)
        data_modified[districts_to_complete_ids['champhai']].append(key)
        data_modified[districts_to_complete_ids['khawzawl']].append(key)
        data_modified[districts_to_complete_ids['serchhip']].append(key)
    elif key.split('/')[0]== 'champhai':
        for key2 in data_modified[key]:
            if key2.split('/')[0] == 'aizwal':
                data_modified[key].remove(key2)
    elif key.split('/')[0]== 'yanam':
        data_modified[key].append(districts_to_complete_ids['east godavari'])
        data_modified[districts_to_complete_ids['east godavari']].append(key)
    elif key.split('/')[0]== 'tenkasi':
        data_modified[key].append(districts_to_complete_ids['tirunelveli'])
        data_modified[key].append(districts_to_complete_ids['virudhunagar'])
        data_modified[key].append(districts_to_complete_ids['kollam'])
        data_modified[key].append(districts_to_complete_ids['thiruvananthapuram'])
        data_modified[key].append(districts_to_complete_ids['thoothukkudi'])
        data_modified[districts_to_complete_ids['tirunelveli']].append(key)
        data_modified[districts_to_complete_ids['virudhunagar']].append(key)
        data_modified[districts_to_complete_ids['kollam']].append(key)
        data_modified[districts_to_complete_ids['thiruvananthapuram']].append(key)
        data_modified[districts_to_complete_ids['thoothukkudi']].append(key)   
    elif key.split('/')[0]== 'tirunelveli':
        for key2 in data_modified[key]:
            if key2.split('/')[0] == 'idukki' or key2.split('/')[0] == 'virudhunagar' or key2.split('/')[0] == 'kollam' or key2.split('/')[0] == 'thiruvananthapuram':
                data_modified[key].remove(key2)
                data_modified[key2].remove(key)
    elif key.split('/')[0]== 'mumbai':
        data_modified[key].append(districts_to_complete_ids['thane'])
        data_modified[districts_to_complete_ids['thane']].append(key)
    elif key.split('/')[0]== 'delhi':
        data_modified[key].append(districts_to_complete_ids['ghaziabad'])
        data_modified[key].append(districts_to_complete_ids['gautam buddha nagar'])
        data_modified[key].append(districts_to_complete_ids['gurugram'])
        data_modified[key].append(districts_to_complete_ids['sonipat'])
        data_modified[key].append(districts_to_complete_ids['baghpat'])
        data_modified[key].append(districts_to_complete_ids['jhajjar'])
        data_modified[key].append(districts_to_complete_ids['faridabad'])
        data_modified[districts_to_complete_ids['ghaziabad']].append(key)
        data_modified[districts_to_complete_ids['gautam buddha nagar']].append(key)
        data_modified[districts_to_complete_ids['gurugram']].append(key)
        data_modified[districts_to_complete_ids['sonipat']].append(key)
        data_modified[districts_to_complete_ids['baghpat']].append(key)
        data_modified[districts_to_complete_ids['jhajjar']].append(key)
        data_modified[districts_to_complete_ids['faridabad']].append(key)   
    data_modified[key].sort()


In [9]:
with open("neighbor-districts-modified.json", "w") as outfile:  
    json.dump(data_modified, outfile, indent = 4) 

In [10]:
# sorting districts alphabetically
districts_from_json = []
for i in data_modified:
    districts_from_json.append(i)
districts_from_json = np.array(districts_from_json)
districts_from_json.sort()

# ids for districts
ids = np.arange(101,101+len(districts_from_json))

# Dictionary for storing ids of districts
Dict = {} 
for i in range(len(ids)):
    Dict[districts_from_json[i]]=ids[i]

data_ids = {}

for i in range(len(ids)):
    data_ids[districts_from_json[i].split('/')[0] + '/' + str(ids[i])] = data_modified[districts_from_json[i]].copy()

for key in data_ids.keys():
    for j in range(len(data_ids[key])):
        curr_id = Dict[data_ids[key][j]]
        data_ids[key][j] = data_ids[key][j].split('/')[0] + '/' + str(curr_id)
# for key in data_modified.keys():
    

In [11]:
with open("neighbor-districts-modified.json", "w") as outfile:  
    json.dump(data_ids, outfile, indent = 4) 